# Часть 1 Бустинг (5 баллов)

В этой части будем предсказывать зарплату data scientist-ов в зависимости  от ряда факторов с помощью градиентного бустинга.

В датасете есть следующие признаки:



* work_year: The number of years of work experience in the field of data science.

* experience_level: The level of experience, such as Junior, Senior, or Lead.

* employment_type: The type of employment, such as Full-time or Contract.

* job_title: The specific job title or role, such as Data Analyst or Data Scientist.

* salary: The salary amount for the given job.

* salary_currency: The currency in which the salary is denoted.

* salary_in_usd: The equivalent salary amount converted to US dollars (USD) for comparison purposes.

* employee_residence: The country or region where the employee resides.

* remote_ratio: The percentage of remote work offered in the job.

* company_location: The location of the company or organization.

* company_size: The company's size is categorized as Small, Medium, or Large.

In [57]:
import pandas as pd

df = pd.read_csv("ds_salaries.csv")
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M


## Задание 1 (0.5 балла) Подготовка



*   Разделите выборку на train, val, test (80%, 10%, 10%)
*   Выберите salary_in_usd в качестве таргета
*   Найдите и удалите признак, из-за которого возможен лик в данных


In [58]:
from sklearn.model_selection import train_test_split
df = df.drop(columns = ['salary'])
X = df.drop(columns=['salary_in_usd'])
y = df['salary_in_usd']

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


Удалил столбец salary, так как по нему скорее всего возможен лик, так как почти 90 процентов значений совпадают в нем и нашем таргете, машина просто бы завышала точность, не выкинув бы мы этот столбец. 

## Задание 2 (0.5 балла) Линейная модель


*   Закодируйте категориальные  признаки с помощью OneHotEncoder
*   Обучите модель линейной регрессии
*   Оцените  качество через MAPE и RMSE


In [61]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV


categorical_features = X_train.select_dtypes(include=['object']).columns
preprocessor = ColumnTransformer(
    transformers=[
        ('categorical', OneHotEncoder(handle_unknown='ignore'), categorical_features)],
    remainder='passthrough')

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('scaler', StandardScaler(with_mean=False)),
    ('model', LinearRegression())
])
pipeline.fit(X_train, y_train)
y_val_pred = pipeline.predict(X_val)
y_test_pred = pipeline.predict(X_test)
mape_test = mean_absolute_percentage_error(y_test, y_test_pred)
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
print(f'MAPE (Test): {mape_test * 100:.2f}%')
print(f'RMSE (Test): {rmse_test:.2f}')

MAPE (Test): 37.14%
RMSE (Test): 51557.01


В целом, предсказание ошибается в ту или иную сторону на 37 процентов (а до стандартизации было 50 %). Но другого и не ждем от простой линейной модели. Зато в будущем будем знать, что по этим метрикам новые, более сложные модели, точно должны быть лучше (меньше в значении).

## Задание 3 (0.5 балла) XGboost

Начнем с библиотеки xgboost.

Обучите модель `XGBRegressor` на тех же данных, что линейную модель, подобрав оптимальные гиперпараметры (`max_depth, learning_rate, n_estimators, gamma`, etc.) по валидационной выборке. Оцените качество итоговой модели (MAPE, RMSE), скорость обучения и скорость предсказания.

<span style="color: red; font-size: 25px;">Я не буду использовать стандартный перебор фором по всем парамерам, а воспользуюсь благами цивилизации и возьму "GridSearchCV". Так быстрей и большее кол-во параметров сможем рассмотреть, а значит, повысится и точность.</span>

In [60]:
import time
from xgboost import XGBRegressor
from IPython.display import Markdown, display

params = {
    'model__max_depth': [3, 5, 7],
    'model__learning_rate': [0.01, 0.1, 0.2],
    'model__n_estimators': [100, 200, 300],
    'model__gamma': [0, 0.1, 0.3],
    'model__colsample_bytree': [0.7, 0.8, 1.0],
    'model__subsample': [0.7, 0.8, 1.0]
}
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('scaler', StandardScaler(with_mean=False)),
    ('model', XGBRegressor())
])
grid_search = GridSearchCV(pipeline, params, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)
start_time = time.time()
grid_search.fit(X_train, y_train)
training_time = time.time() - start_time

start_time = time.time()
y_test_pred = best_model.predict(X_test)
prediction_time_test = time.time() - start_time

mape_test = mean_absolute_percentage_error(y_test, y_test_pred)
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
def print_params(params):
    md = "### Best Model Parameters\n"
    for key, value in params.items():
        md += f"- **{key.split('__')[-1].replace('_', ' ').title()}**: {value}\n"
    display(Markdown(md))

print_params(best_params)
print(
    f'training_time - {training_time:.2f}',
    f'\nprediction_time_test - {prediction_time_test}',
    f'\nmape_test - {mape_test * 100:.2f}%',
    f'\nrmse_test - {rmse_test:.2f}'
)

### Best Model Parameters
- **Colsample Bytree**: 0.8
- **Gamma**: 0
- **Learning Rate**: 0.1
- **Max Depth**: 5
- **N Estimators**: 100
- **Subsample**: 0.8


training_time - 21.25 
prediction_time_val - 0.002855062484741211 
prediction_time_test - 0.0024220943450927734 
mape_test - 34.38% 
rmse_test - 50408.72


Как мы видим, времени на обучение заняло больше, но и ошибка уменьшилась, а значит, все не зря.

## Задание 4 (1 балл) CatBoost

Теперь библиотека CatBoost.

Обучите модель `CatBoostRegressor`, подобрав оптимальные гиперпараметры (`depth, learning_rate, iterations`, etc.) по валидационной выборке. Оцените качество итоговой модели (MAPE, RMSE), скорость обучения и скорость предсказания.

In [63]:
from catboost import CatBoostRegressor, Pool

params = {
    'model__depth': [4, 6, 8, 10],
    'model__learning_rate': [0.01, 0.1, 0.2],
    'model__iterations': [200, 500, 1000],
    'model__l2_leaf_reg': [1, 3, 5],
    'model__subsample': [0.8, 1.0]
}
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('scaler', StandardScaler(with_mean=False)),
    ('model', CatBoostRegressor(verbose=0))
])
grid_search = GridSearchCV(pipeline, params, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)
start_time = time.time()
grid_search.fit(X_train, y_train)
training_time = time.time() - start_time

start_time = time.time()
y_test_pred = best_model.predict(X_test)
prediction_time_test = time.time() - start_time

mape_test = mean_absolute_percentage_error(y_test, y_test_pred)
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))

def print_params(params):
    md = "### Best Model Parameters\n"
    for key, value in params.items():
        md += f"- **{key.split('__')[-1].replace('_', ' ').title()}**: {value}\n"
    display(Markdown(md))

best_params = grid_search.best_params_
print_params(best_params)

print(
    f'training_time - {training_time:.2f}',
    f'\nprediction_time_test - {prediction_time_test:}',
    f'\nmape_test - {mape_test * 100:.2f}%',
    f'\nrmse_test - {rmse_test:.2f}'
)

### Best Model Parameters
- **Depth**: 8
- **Iterations**: 200
- **L2 Leaf Reg**: 5
- **Learning Rate**: 0.1
- **Subsample**: 1.0


training_time - 95.91 
prediction_time_test - 0.003941059112548828 
mape_test - 34.28% 
rmse_test - 49634.26


Еще лучше! Хоть и в 5 раз дольше. 

Для применения catboost моделей не обязательно сначала кодировать категориальные признаки, модель может кодировать их сама. Обучите catboost с подбором оптимальных гиперпараметров снова, используя pool для передачи данных в модель с указанием какие признаки категориальные, а какие нет с помощью параметра cat_features. Оцените качество и время. Стало ли лучше?

In [21]:
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M


Не получилось адаптировать GridSearchCV для этой задачи, а простым перебором слищком долго и тогда нечестное сравнение методов. Но попробую в дальнейшем добрать баллы с помощью бонусных заданий.

## Задание 5 (0.5 балла) LightGBM

И наконец библиотека LightGBM - используйте `LGBMRegressor`, снова подберите гиперпараметры, оцените качество и скорость.


In [78]:
from lightgbm import LGBMRegressor
import scipy.stats as st
from sklearn.model_selection import RandomizedSearchCV

params = {
    'model__num_leaves': st.randint(20, 50),
    'model__learning_rate': st.uniform(0.01, 0.2),
    'model__n_estimators': st.randint(100, 1000),
    'model__min_child_weight': st.randint(1, 6),
    'model__subsample': st.uniform(0.8, 1.0)
}

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('scaler', StandardScaler(with_mean=False)),
    ('model', LGBMRegressor(verbosity=-1))
])

random_search = RandomizedSearchCV(pipeline, params, n_iter=50, cv=3, scoring='neg_mean_squared_error', n_jobs=-1, random_state=42)
start_time = time.time()
random_search.fit(X_train, y_train)
training_time = time.time() - start_time

best_model = random_search.best_estimator_

start_time = time.time()
y_test_pred = best_model.predict(X_test)
prediction_time_test = time.time() - start_time

mape_test = mean_absolute_percentage_error(y_test, y_test_pred)
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))

def print_params(params):
    md = "### Best Model Parameters\n"
    for key, value in params.items():
        md += f"- **{key.split('__')[-1].replace('_', ' ').title()}**: {value}\n"
    display(Markdown(md))

best_params = random_search.best_params_
print_params(best_params)

print(
    f'training_time - {training_time:.2f} seconds',
    f'\nprediction_time_test - {prediction_time_test:.2f} seconds',
    f'\nmape_test - {mape_test * 100:.2f}%',
    f'\nrmse_test - {rmse_test:.2f}'
)

[LightGBM] [Fatal] Check failed: (bagging_fraction) <= (1.0) at /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_7979061qoc/croot/lightgbm_1714113233928/work/src/io/config_auto.cpp, line 365 .

[LightGBM] [Fatal] Check failed: (bagging_fraction) <= (1.0) at /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_7979061qoc/croot/lightgbm_1714113233928/work/src/io/config_auto.cpp, line 365 .

[LightGBM] [Fatal] Check failed: (bagging_fraction) <= (1.0) at /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_7979061qoc/croot/lightgbm_1714113233928/work/src/io/config_auto.cpp, line 365 .

[LightGBM] [Fatal] Check failed: (bagging_fraction) <= (1.0) at /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_7979061qoc/croot/lightgbm_1714113233928/work/src/io/config_auto.cpp, line 365 .

[LightGBM] [Fatal] Check failed: (bagging_fraction) <= (1.0) at /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_7979061qoc/croot/lightgbm_1714113233928/work/src/io

### Best Model Parameters
- **Learning Rate**: 0.010104075399063163
- **Min Child Weight**: 5
- **N Estimators**: 844
- **Num Leaves**: 22
- **Subsample**: 0.8069521305311907


training_time - 149.28 seconds 
prediction_time_test - 0.01 seconds 
mape_test - 32.04% 
rmse_test - 49747.86


## Задание 6 (2 балла) Сравнение и выводы

Сравните модели бустинга и сделайте про них выводы, какая из моделей показала лучший/худший результат по качеству, скорости обучения и скорости предсказания? Как отличаются гиперпараметры для разных моделей?

### Сравнение моделей:

- **Линейная регрессия**:
  - MAPE: 37.14%
  - RMSE: 51557.01
  - Время обучения: мгновенно

- **XGBoost**:
  - MAPE: 34.38%
  - RMSE: 50408.72
  - Время обучения: 21.25 секунд
  - Время предсказания: 0.0024 секунд

- **CatBoost**:
  - MAPE: 34.28%
  - RMSE: 49634.26
  - Время обучения: 95.91 секунд
  - Время предсказания: 0.0039 секунд

- **LightGBM**:
  - MAPE: 32.04%
  - RMSE: 49747.86
  - Время обучения: 149.28 секунд
  - Время предсказания: 0.01 секунд

### Выводы:
- **Качество (MAPE, RMSE)**: LightGBM лучший, линейная регрессия худшая.
- **Скорость обучения**: Линейная регрессия самая быстрая, XGBoost быстрее остальных бустинговых моделей.
- **Скорость предсказания**: Линейная регрессия и XGBoost самые быстрые.

Для быстрого обучения и предсказания лучше использовать линейную регрессию или XGBoost. Для лучшего качества предсказаний - LightGBM.

# Часть 2 Кластеризация (5 баллов)

Будем работать с данными о том, каких исполнителей слушают пользователи музыкального сервиса.

Каждая строка таблицы - информация об одном пользователе. Каждый столбец - это исполнитель (The Beatles, Radiohead, etc.)

Для каждой пары (пользователь, исполнитель) в таблице стоит число - доля прослушивания этого исполнителя этим пользователем.


In [79]:
import pandas as pd
ratings = pd.read_excel("https://github.com/evgpat/edu_stepik_rec_sys/blob/main/datasets/sample_matrix.xlsx?raw=true", engine='openpyxl')
ratings.head()

,user,the beatles,radiohead,deathcab for cutie,coldplay,modest mouse,sufjan stevens,dylan. bob,red hot clili peppers,pink fluid,...,municipal waste,townes van zandt,curtis mayfield,jewel,lamb,michal w. smith,群星,agalloch,meshuggah,yellowcard
0,0,NaN,0.020417,NaN,NaN,NaN,NaN,NaN,0.030496,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,0.184962,0.024561,NaN,NaN,0.136341,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,0.028635,NaN,NaN,NaN,0.024559,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0.043529,0.086281,0.034590,0.016712,0.015935,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Будем строить кластеризацию исполнителей: если двух исполнителей слушало много людей примерно одинаковую долю своего времени (то есть векторы близки в пространстве), то, возможно исполнители похожи. Эта информация может быть полезна при построении рекомендательных систем.

In [102]:
df_first = ratings

## Задание 1 (0.5 балла) Подготовка

Транспонируем матрицу ratings, чтобы по строкам стояли исполнители.

In [103]:
ratings = ratings.set_index('user').transpose()

Выкиньте строку под названием `user`.

In [104]:
ratings = ratings.drop('user', errors='ignore')

В таблице много пропусков, так как пользователи слушают не всех-всех исполнителей, чья музыка представлена в сервисе, а некоторое подмножество (обычно около 30 исполнителей)


Доля исполнителя в музыке, прослушанной  пользователем, равна 0, если пользователь никогда не слушал музыку данного музыканта, поэтому заполните пропуски нулями.



In [105]:
ratings = ratings.fillna(0)
ratings.sample()

user,0,1,2,3,4,5,6,7,8,9,...,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
fear factory,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Задание 2 (0.5 балла) Первая кластеризация

Примените KMeans с 5ю кластерами, сохраните полученные лейблы

In [106]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=5, random_state=0)
kmeans.fit(ratings)
labels = kmeans.labels_

/opt/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Выведите размеры кластеров. Полезной ли получилась кластеризация? Почему KMeans может выдать такой результат?

In [107]:
print(np.bincount(labels))

[996   1   1   1   1]


**Ответ:** Таблица имеет много пропусков (NaN), что указывает на то, что большинство пользователей слушают ограниченное количество исполнителей, и так как большинство значений в таблице равны нулю (что означает, что пользователь не слушал данного исполнителя), то модели кластеризации будет трудно найти значимые кластеры, по итогу он собрал почти все в один большой.

## Задание 3 (0.5 балла) Объяснение результатов

При кластеризации получилось $\geq 1$ кластера размера 1. Выведите исполнителей, которые составляют такие кластеры. Среди них должна быть группа The Beatles.

In [108]:
one_clust = np.where(np.bincount(labels) == 1)[0]
one_clust = ratings.index[np.isin(labels, one_clust)]
print(one_clust)

Index(['the beatles', 'daft punk', 'morricone', '보아'], dtype='object')


In [109]:
user_counts = (ratings > 0).sum(axis=1) / ratings.shape[1]
print("Доля слушателей, которые слушают каждого исполнителя:")
print(user_counts)
# Рассчитаем среднюю долю прослушивания для каждого исполнителя
mean_listening_shares = ratings.mean(axis=1)
print("Средняя доля прослушивания для каждого исполнителя:")
print(mean_listening_shares)
top_artists_by_users = user_counts.sort_values(ascending=False).head(10)
top_artists_by_listening = mean_listening_shares.sort_values(ascending=False).head(10)
print("Исполнители с самой высокой долей пользователей:")
print(top_artists_by_users)
print("Исполнители с самой высокой средней долей прослушивания:")
print(top_artists_by_listening)

Доля пользователей, которые слушают каждого исполнителя:
the beatles           0.3342
radiohead             0.2778
deathcab for cutie    0.1862
coldplay              0.1682
modest mouse          0.1628
                       ...  
michal w. smith       0.0094
群星                    0.0094
agalloch              0.0094
meshuggah             0.0094
yellowcard            0.0094
Length: 1000, dtype: float64
Средняя доля прослушивания для каждого исполнителя:
the beatles           0.018369
radiohead             0.011851
deathcab for cutie    0.006543
coldplay              0.006030
modest mouse          0.005876
                        ...   
michal w. smith       0.000895
群星                    0.000519
agalloch              0.000997
meshuggah             0.000431
yellowcard            0.000320
Length: 1000, dtype: float64
Исполнители с самой высокой долей пользователей:
the beatles              0.3342
radiohead                0.2778
deathcab for cutie       0.1862
coldplay                 0.1

Изучите данные, почему именно The Beatles выделяется?

Подсказка: посмотрите на долю пользователей, которые слушают каждого исполнителя, среднюю долю прослушивания.

Все логично, так как битлз слушают чаще, чем кого-либо еще, поэтому он попал в отдельный кластер исполнителей.

## Задание 4 (0.5 балла) Улучшение кластеризации

Попытаемся избавиться от этой проблемы: нормализуйте данные при помощи `normalize`.

In [97]:
from sklearn.preprocessing import normalize
ratings = normalize(ratings)
kmeans = KMeans(n_clusters=5, random_state=0)
kmeans.fit(ratings)
labels = kmeans.labels_
print(np.bincount(labels))

/opt/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


[208 129 230  89 344]


Примените KMeans с 5ю кластерами на преобразованной матрице, посмотрите на их размеры. Стало ли лучше? Может ли кластеризация быть полезной теперь?

**Ответ** У нас появилось более равномерное распределение, а значит, у нас есть значимые группы исполнителей. Стало, безусловно, лучше.

## Задание 5 (1 балл) Центроиды

Выведите для каждого кластера названия топ-10 исполнителей, ближайших к центроиду по косинусной мере. Проинтерпретируйте результат. Что можно сказать о смысле кластеров?

In [ ]:
from scipy.spatial.distance import cosine


centroids = km.cluster_centers_

# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 6 (1 балл) Визуализация

Хотелось бы как-то визуализировать полученную кластеризацию. Постройте точечные графики `plt.scatter` для нескольких пар признаков исполнителей, покрасив точки в цвета кластеров. Почему визуализации получились такими? Хорошо ли они отражают разделение на кластеры? Почему?

In [ ]:
import matplotlib.pyplot as plt

# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Для визуализации данных высокой размерности существует метод t-SNE (стохастическое вложение соседей с t-распределением). Данный метод является нелинейным методом снижения размерности: каждый объект высокой размерности будет моделироваться объектов более низкой (например, 2) размерности таким образом, чтобы похожие объекты моделировались близкими, непохожие - далекими с большой вероятностью.

Примените `TSNE` из библиотеки `sklearn` и визуализируйте полученные объекты, покрасив их в цвета их кластеров

In [ ]:
from sklearn.manifold import TSNE

# -- YOUR CODE HERE --

## Задание 7 (1 балл) Подбор гиперпараметров

Подберите оптимальное количество кластеров (максимум 100 кластеров) с использованием индекса Силуэта. Зафиксируйте `random_state=42`

In [ ]:
from sklearn.metrics import silhouette_score

# -- YOUR CODE HERE --

Выведите исполнителей, ближайших с центроидам (аналогично заданию 5). Как соотносятся результаты? Остался ли смысл кластеров прежним? Расскажите про смысл 1-2 интересных кластеров, если он изменился и кластеров слишком много, чтобы рассказать про все.

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Сделайте t-SNE визуализацию полученной кластеризации.

In [ ]:
# -- YOUR CODE HERE --

Если кластеров получилось слишком много и визуально цвета плохо отличаются, покрасьте только какой-нибудь интересный кластер из задания выше (`c = (labels == i)`). Хорошо ли этот кластер отражается в визуализации?

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --